In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1611087723218_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Query some reviews
query_1 = "select distinct r.ugc_id,  mg.name, r.review_headline, r.review_comments, r.review_length, r.review_rating, r.review_would_recommend, \
case when (contains(r.observations, 'CC') )  then 1 else 0 end as CC \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where r.locale in ('en_US','en_CA','en_GB') and contains(r.observations, 'PM') and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and r.created_date > '2020-11-01' \
"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
reviews = pd.read_sql(query_1, conn)
reviews.head()
#reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID        NAME  ... REVIEW_WOULD_RECOMMEND CC
0  324547054  Colgate US  ...                   True  0
1  324547450  Colgate US  ...                   True  0
2  324551754  Colgate US  ...                   True  0
3  324556165  Colgate US  ...                   True  0
4  324559748  Colgate US  ...                   True  0

[5 rows x 8 columns]

In [72]:
reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + '. '+reviews['REVIEW_COMMENTS']
#Use set_option so text doesn't truncate

#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#reviews.head()

#reviews.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2011193 entries, 0 to 2011808
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   UGC_ID                  int64  
 1   NAME                    object 
 2   REVIEW_HEADLINE         object 
 3   REVIEW_COMMENTS         object 
 4   REVIEW_LENGTH           float64
 5   REVIEW_RATING           int64  
 6   REVIEW_WOULD_RECOMMEND  object 
 7   CC                      int64  
 8   ALL_TEXT                object 
 9   sentiment_score         float64
 10  model_CC                int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 184.1+ MB

In [74]:
#Remove nulls so sentiment analyzer works
reviews=reviews[reviews['ALL_TEXT'].notnull()]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
#Run sentiment on headline and comments
#Compare to star rating
#Compare to bottom line/would recommend Y/N

sentiment_analyzer = SentimentIntensityAnalyzer()
reviews['sentiment_score'] = reviews['ALL_TEXT'].apply(lambda x: sentiment_analyzer.polarity_scores(x)['compound'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.

In [77]:
reviews['model_CC'] = 0
#reviews.loc[( ((reviews['REVIEW_RATING']==5) | (reviews['REVIEW_RATING']==4)) 
#             & ((reviews['sentiment_score']<-0.7) | (reviews['REVIEW_WOULD_RECOMMEND']==False))  ) |
#            ( ((reviews['REVIEW_RATING']==1) | (reviews['REVIEW_RATING']==2)) 
#             & ((reviews['sentiment_score']>0.7) | (reviews['REVIEW_WOULD_RECOMMEND']==True)) ), 'model_CC'] = 1


reviews.loc[( ((reviews['REVIEW_RATING']==5) | (reviews['REVIEW_RATING']==4)) & (reviews['REVIEW_WOULD_RECOMMEND']==False)  ) |
            ( ((reviews['REVIEW_RATING']==1) | (reviews['REVIEW_RATING']==2)) & (reviews['REVIEW_WOULD_RECOMMEND']==True) ) |
        (((reviews['REVIEW_RATING']==5) | (reviews['REVIEW_RATING']==4)) & (reviews['sentiment_score']<-0.75) & (reviews['REVIEW_LENGTH']<20) )  |
        (((reviews['REVIEW_RATING']==1) | (reviews['REVIEW_RATING']==2)) & (reviews['sentiment_score']>0.75) & (reviews['REVIEW_LENGTH']<20) )  , 'model_CC'] = 1


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
reviews['model_CC'] = 0

reviews.loc[( ((reviews['REVIEW_RATING']==5) | (reviews['REVIEW_RATING']==4)) & (reviews['REVIEW_WOULD_RECOMMEND']==False)  ) |
            ( ((reviews['REVIEW_RATING']==1) | (reviews['REVIEW_RATING']==2)) & (reviews['REVIEW_WOULD_RECOMMEND']==True) ) , 'model_CC'] = 1


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID        NAME                         REVIEW_HEADLINE  \
0  324547054  Colgate US  Colgate Renewal Gel Toothpaste review    
1  324547450  Colgate US  Love the mint flavor!                    
2  324551754  Colgate US  GOING WHITE AND NOT COMING BACK.         
3  324556165  Colgate US  Colgate Renewal Whitening Restoration.   
4  324559748  Colgate US  Fresh & Restore Whitening                

                                                                                                                                                                                                                                                                                                                                                                                                                                                                  REVIEW_COMMENTS  \
0  I really loved this toothpaste and I am so happy that I was able to test it.  I love the cool mint flavor which leaves 

In [79]:
FP = reviews[(reviews['CC'] == 0) & (reviews['model_CC'] == 1)]
FN = reviews[(reviews['CC'] == 1) & (reviews['model_CC'] == 0)]
TP = reviews[(reviews['CC'] == 1) & (reviews['model_CC'] == 1)]
TN = reviews[(reviews['CC'] == 0) & (reviews['model_CC'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13671

In [81]:
FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6572

In [82]:
TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1183

In [83]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1989767

In [84]:
FP.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          UGC_ID                      NAME  \
58     324532861  Salsas.com                 
204    324504230  JD Sports Fashion UK       
332    324508978  JD Sports Fashion UK       
356    324543238  JD Sports Fashion UK       
393    324504138  JD Sports Fashion UK       
401    324485095  JD Sports Fashion UK       
463    322275550  Hayneedle.com              
554    322277097  Hayneedle.com              
559    322270395  Hayneedle.com              
666    322252250  UncommonGoods              
953    320139835  The Company Store          
1021   320036013  The Company Store          
1034   319960426  Party City, Inc.           
1153   319844646  Party City, Inc.           
1370   319863326  Shutterfly                 
1371   320018545  Shutterfly                 
1403   322261285  Ulta                       
1603   322281699  Timberland                 
1649   322268393  TSC                        
1654   322261266  TSC                        
1688   322245257  BJ's Wholesale C

In [85]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("NAME", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("REVIEW_LENGTH", FloatType(), True)\
,StructField("REVIEW_RATING", StringType(), True)\
,StructField("REVIEW_WOULD_RECOMMEND", BooleanType(), True)\
,StructField("CC", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("sentiment_score", FloatType(), True)\
,StructField("model_CC", IntegerType(), True)])   



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FP = spark.createDataFrame(FP, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
FP.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","CC_FP_test5_sent75_char20").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_test1")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_test1")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…